In [1]:
#import dependancies
import pandas as pd
from sqlalchemy import create_engine
from config import dbConnectionString
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
engine = create_engine(dbConnectionString)

#import dataframe from SQL database
df = pd.read_sql_query('select * from "cardioData"',con=engine)
df = df.drop(columns = ['index', 'id'])
df

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [3]:
#generate label encoder
le = LabelEncoder()
df2 = df.copy()
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [4]:
df2['BMI'] = df2['BMI'].astype(float)

In [5]:
#bucket bmi and blood pressure
df2['BMI_class'] = ''

for i in range(len(df2['age'])):
    if df2['BMI'][i] < 18.5:
        df2['BMI_class'][i] = 'underweight'
    elif df2['BMI'][i] < 24.9 and df2['BMI'][i] > 18.5:
        df2['BMI_class'][i] = 'normal'
    elif df2['BMI'][i] < 29.9 and df2['BMI'][i] > 24.9:
        df2['BMI_class'][i] = 'overweight'
    elif df2['BMI'][i] < 34.9 and df2['BMI'][i] > 29.9:
        df2['BMI_class'][i] = 'obese'
    elif df2['BMI'][i] < 39.9 and df2['BMI'][i] > 34.9:
        df2['BMI_class'][i] = 'obese2'
    else:
        df2['BMI_class'][i] = 'obese3'
        
df2['ap_hi_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_hi'][i] < 120:
        df2['ap_hi_class'][i] = 'normal'
    elif df2['ap_hi'][i] < 129 and df2['ap_hi'][i] >= 120:
        df2['ap_hi_class'][i] = 'elevated'
    elif df2['ap_hi'][i] < 139 and df2['ap_hi'][i] >= 130:
        df2['ap_hi_class'][i] = 'highbloodpressure'
    elif df2['ap_hi'][i] < 180 and df2['ap_hi'][i] >= 140:
        df2['ap_hi_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_hi_class'][i] = 'hypertensive'
        
df2['ap_lo_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_lo'][i] < 80:
        df2['ap_lo_class'][i] = 'elevated'
    elif df2['ap_lo'][i] < 80 and df2['ap_lo'][i] >= 89:
        df2['ap_lo_class'][i] = 'highbloodpressure'
    elif df2['ap_lo'][i] < 120 and df2['ap_lo'][i] >= 90:
        df2['ap_lo_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_lo_class'][i] = 'hypertensive'
        
df2

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,Female,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,Male,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,Male,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,Male,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,Male,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,Male,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,Female,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,Male,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [6]:
#encode gender: female = 0, male = 1
df2['gender'] = le.fit_transform(df2['gender'])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,1,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,1,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,1,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,1,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,1,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,0,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,1,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [7]:
#encode all other features
normal_dic = {
    "normal" : 0,
    "above normal" : 1,
    "well above normal" : 2,
}

true_false_dic = {
    True : 1,
    False : 0,
}

BMI_dic = {
    "underweight" : 0,
    "normal" : 1,
    "overweight" : 2, 
    "obese": 3, 
    "obese2" : 4, 
    "obese3" : 5
}

blood_pressure_dic = {
    "normal" : 0,
    "elevated" : 1, 
    "highbloodpressure": 2, 
    "highbloodpressure2": 3,
    "hypertensive" : 4
}

df2['cardiovascular'] = df2['cardiovascular'].apply(lambda x : true_false_dic[x])
df2['smoke'] = df2['smoke'].apply(lambda x : true_false_dic[x])
df2['alchohol'] = df2['alchohol'].apply(lambda x : true_false_dic[x])
df2['active'] = df2['active'].apply(lambda x : true_false_dic[x])

df2['cholestrol'] = df2['cholestrol'].apply(lambda x : normal_dic[x])
df2['gloucose'] = df2['gloucose'].apply(lambda x : normal_dic[x])
df2['BMI_class'] = df2['BMI_class'].apply(lambda x : BMI_dic[x])
df2['ap_hi_class'] = df2['ap_hi_class'].apply(lambda x : blood_pressure_dic[x])
df2['ap_lo_class'] = df2['ap_lo_class'].apply(lambda x : blood_pressure_dic[x])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,0,23.0,110,80,0,0,0,0,1,1,0,4
1,55.4,1,156.0,85.0,1,46.0,140,90,2,0,0,0,1,5,3,3
2,51.7,1,165.0,64.0,1,25.0,130,70,2,0,0,0,0,2,2,1
3,47.9,1,156.0,56.0,0,20.0,100,60,0,0,0,0,0,1,0,1
4,60.0,1,151.0,67.0,0,30.0,120,80,1,1,0,0,0,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,0,34.0,120,80,0,0,1,0,1,3,1,4
63912,61.9,1,158.0,126.0,1,100.0,140,90,1,1,0,0,1,5,3,3
63913,52.2,0,183.0,105.0,1,60.0,180,90,2,0,0,1,0,5,4,3
63914,61.5,1,163.0,72.0,1,32.0,135,80,0,2,0,0,0,3,2,4


In [9]:
# set X and y values
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [10]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

#scale data
scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
#generate decision tree model
model = tree.DecisionTreeClassifier()

In [12]:
model = model.fit(X_train_scaled, y_train)

In [13]:
#generate confusion matrix
predictions = model.predict(X_test_scaled)
confusion_matrix(y_test, predictions)

array([[5426, 3057],
       [2959, 4537]], dtype=int64)

In [14]:
#generate accuracy score
accuracy_score(y_test, predictions)

0.6235058514300019

In [15]:
#BMI isntead of height/weight
X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)

accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


0.6345828900431817

In [16]:
# bucket bmi
X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)

accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


0.6531697853432631

In [17]:
#bucket blood pressure
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)

accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


0.6232555228737718

In [18]:
# bucket both

X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1).values
y = df2['cardiovascular']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)

accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


0.6578634457725765

## Random Forest

In [44]:
#original dataset
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [46]:
rf_model = RandomForestClassifier(n_estimators = 128, random_state = 9)

In [47]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [48]:
predictions = rf_model.predict(X_test_scaled)

In [49]:
confusion_matrix(y_test, predictions)

array([[6212, 2271],
       [2461, 5035]], dtype=int64)

In [50]:
accuracy_score(y_test, predictions)

0.7038613179798485

In [51]:
#increase estimators
rf_model = RandomForestClassifier(n_estimators = 500, random_state = 9)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

0.7034858251455035

In [52]:
importances = rf_model.feature_importances_
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1)
sorted(zip(importances, X.columns), reverse = True)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


[(0.2806797998705362, 'age'),
 (0.19999771563656848, 'weight'),
 (0.18188470458147915, 'height'),
 (0.16317372400396282, 'ap_hi'),
 (0.07120740067567906, 'ap_lo'),
 (0.03877620121202788, 'cholestrol'),
 (0.017674851901421092, 'gender'),
 (0.014981693903398211, 'gloucose'),
 (0.014172462679451898, 'active'),
 (0.009421060644468232, 'smoke'),
 (0.008030384891007028, 'alchohol')]

In [27]:
#test using only top features

# set X and y values
X = df2.drop(['cardiovascular', 'BMI', 'gender', 'gloucose', 'active', 'smoke', 'alchohol', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [28]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
#increase estimators
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 9)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

0.6916578008636335

In [34]:
# use bmi 

X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators = 200, random_state = 9)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


0.6837724513423868

In [35]:
#bucket bmi

X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators = 200, random_state = 9)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


0.6807059265285688

In [38]:
#bucket blood pressure
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators = 200, random_state = 9)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


0.7031729144502159

In [39]:
importances = rf_model.feature_importances_
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1)
sorted(zip(importances, X.columns), reverse = True)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


[(0.28065014404126404, 'age'),
 (0.19955183542802826, 'weight'),
 (0.1819307450766764, 'height'),
 (0.1644654124706611, 'ap_hi'),
 (0.07045356041497065, 'ap_lo'),
 (0.03869943175294215, 'cholestrol'),
 (0.01756515722550207, 'gender'),
 (0.014771641585694648, 'gloucose'),
 (0.014309404164303456, 'active'),
 (0.009489490474140034, 'smoke'),
 (0.008113177365817223, 'alchohol')]

In [37]:
# bucket both
X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1).values
y = df2['cardiovascular']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators = 200, random_state = 9)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
accuracy_score(y_test, predictions)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


0.6760748482383128